- A informação da coluna `tipo` expressa o tipo de relâmpago
  * tipo=0 - Relâmpago nuvem-solo
  * tipo=1 - Relâmpago intranuvem

# Preparando ambiente

In [ ]:
# importa bibliotecas
import pandas as pd
from datetime import datetime, timedelta
import shutil
import os
from datetime import datetime
import calendar
import gzip
import shutil
import time

# monta drive
from google.colab import drive
drive.mount('/content/drive')

# diretório
dir = '/content/drive/MyDrive/00_DISPONIBILIZAÇÃO_DE_AULAS/1_PRIMEIRO_SEMESTRE/CAT212 - MICROFÍSICA DE NUVENS E ELETRICIDADE ATMOSFÉRICA/MONOGRAFIA/Dados_Monografia'

Mounted at /content/drive


# Testes iniciais

In [ ]:
# ano e mês
ano, mes = '2021', '01'

#  vizualiza apenas as 3 primeiras para escolhermos apenas as colunas de interesse
df_teste = pd.read_csv(f'{dir}/{ano}/raios{ano}{mes}.zip', nrows=3)
df_teste

,tipo,horario,lon_lat,longitude,latitude,pico_de_corrente,ic_height,numero_de_sensores,multiplicidade,the_geom,majorerrorelipse,minorerrorelipse,bearingerrorelipse
0,1,2021-01-01 00:00:00.272114,"(-49.27115,-27.1948)",-49.27115,-27.19480,2571,15724,6,NaN,0101000020E6100000BB270F0BB5A248C0C442AD69DE31...,265.0,151.0,153.0
1,1,2021-01-01 00:00:00.418449,"(-38.27389,-32.2791799)",-38.27389,-32.27918,-9879,15949,11,NaN,0101000020E6100000C959D8D30E2343C0AC19BE2ABC23...,157.0,125.0,13.0
2,0,2021-01-01 00:00:00.70984,"(-39.93473,-30.41494)",-39.93473,-30.41494,-34768,0,16,NaN,0101000020E6100000884B8E3BA5F743C060CD0182396A...,675.0,651.0,73.0


In [ ]:
df_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tipo                3 non-null      int64  
 1   horario             3 non-null      object 
 2   lon_lat             3 non-null      object 
 3   longitude           3 non-null      float64
 4   latitude            3 non-null      float64
 5   pico_de_corrente    3 non-null      int64  
 6   ic_height           3 non-null      int64  
 7   numero_de_sensores  3 non-null      int64  
 8   multiplicidade      0 non-null      float64
 9   the_geom            3 non-null      object 
 10  majorerrorelipse    3 non-null      float64
 11  minorerrorelipse    3 non-null      float64
 12  bearingerrorelipse  3 non-null      float64
dtypes: float64(6), int64(4), object(3)
memory usage: 444.0+ bytes


# Lendo o arquivo completo:
- A leitura direta consome mais mémoria RAM = 5.0 Gb

In [ ]:
%%time
# ano e mês
ano, mes = '2021', '01'

# leitura: vamos utilizar apenas as seguintes colunas: 'tipo','horario','longitude','latitude'
df = pd.read_csv(f'{dir}/{ano}/raios{ano}{mes}.zip', usecols=['tipo','horario','longitude','latitude'])

# mostra o dado
df

CPU times: user 1min 50s, sys: 8.13 s, total: 1min 58s
Wall time: 2min 15s


,tipo,horario,longitude,latitude
0,1,2021-01-01 00:00:00.272114,-49.27115,-27.19480
1,1,2021-01-01 00:00:00.418449,-38.27389,-32.27918
2,0,2021-01-01 00:00:00.70984,-39.93473,-30.41494
3,1,2021-01-01 00:00:01.283056,-44.58479,-26.41747
4,1,2021-01-01 00:00:01.317802,-67.33449,-25.34216
...,...,...,...,...
36110770,1,2021-01-31 23:59:58.281036,-51.27836,-18.72863
36110771,1,2021-01-31 23:59:58.457549,-51.27755,-18.74969
36110772,1,2021-01-31 23:59:58.524168,-51.26730,-18.73606
36110773,1,2021-01-31 23:59:58.8052,-46.09995,-11.48831


# Lendo o arquivo em pedaços com chunksize (streaming):
- A leitura em chunks consome menos mémoria RAM = 1.9 Gb

In [ ]:
%%time

# limites de MG
lonmin_MG, lonmax_MG = -52., -39.
latmin_MG, latmax_MG = -23., -14.

# ano e mês
ano, mes = '2021', '01'

# tamanho do chunks
chunks = []  # lista para acumular os DataFrames filtrados
chunksize = 500_000

# loop nos pedaços
for chunk in pd.read_csv(f'{dir}/{ano}/raios{ano}{mes}.zip',
                         usecols=['tipo', 'horario', 'longitude', 'latitude'],
                         chunksize=chunksize):

    print(chunk.shape)

    # filtra para a região de estudo
    filtrado = chunk[ (chunk['longitude'] > lonmin_MG) & (chunk['longitude'] < lonmax_MG) &
                      (chunk['latitude'] > latmin_MG)  & (chunk['latitude'] < latmax_MG)]

    chunks.append(filtrado)

# concatena todos os pedaços filtrados
df_tot = pd.concat(chunks, ignore_index=True)

# mostra o dado
df_tot

(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(500000, 4)
(110775, 4)
CPU times: user 1min 52s, sys: 4.33 s, total: 1min 56s
Wall time: 2min 12s


,tipo,horario,longitude,latitude
0,1,2021-01-01 00:00:01.491999,-48.24018,-15.38519
1,1,2021-01-01 00:00:01.497913,-48.24333,-15.38145
2,1,2021-01-01 00:00:03.617269,-43.58005,-20.57816
3,1,2021-01-01 00:00:04.327901,-43.57654,-20.60958
4,1,2021-01-01 00:00:04.447535,-47.79835,-22.43118
...,...,...,...,...
3529672,1,2021-01-31 23:59:58.112825,-51.26455,-18.73600
3529673,1,2021-01-31 23:59:58.281036,-51.27836,-18.72863
3529674,1,2021-01-31 23:59:58.457549,-51.27755,-18.74969
3529675,1,2021-01-31 23:59:58.524168,-51.26730,-18.73606
